In [1]:
#Root 경로로 이동
%cd ..

/Users/shin/MyDir/MyGit/BrainMR_MCI


In [2]:
from model import _CNN
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
#from loss import ConRegGroupLoss
import dataloader
import utils
import pandas as pd
from torch.utils.data import DataLoader


In [3]:
config = utils.load_config()
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
X_train,X_test,y_train,y_test = dataloader.dataset_split(df_dataset)
#y_train = y_train.str.replace('MCI','1').replace('CN','0').replace('AD','2')
#y_test = y_test.str.replace('MCI','1').replace('CN','0').replace('AD','2')

In [4]:
traindata=dataloader.MRIDataset(X_train,y_train)
testdata=dataloader.MRIDataset(X_test,y_test)

train_dataloader = DataLoader(traindata, batch_size=4, shuffle=False)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

In [14]:
device = torch.device('mps')


model = _CNN(fil_num=10, drop_rate=0.137).to(device)
optimizer = optim.Adam(model.parameters(), lr= 0.0001, betas=(0.5, 0.999))
criterion_clf = nn.CrossEntropyLoss(weight=torch.Tensor([1, 3])).to(device)
criterion_reg = nn.SmoothL1Loss(reduction='mean').to(device)

In [15]:

model.train(True)
for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels

    model.zero_grad()
    loss = torch.tensor(0.0, requires_grad=True).to(device)
    clf_output, reg_output, per_loss = model(inputs)
    clf_loss = criterion_clf(clf_output, labels)
    loss = loss + clf_loss
    loss.backward()
    optimizer.step()

KeyboardInterrupt: 

In [ ]:
def train_model_epoch(self):
    self.model.train(True)
    for inputs, labels, demors in self.train_dataloader:
        inputs, labels, demors = inputs.cuda(), labels.cuda(), demors.cuda()
        self.model.zero_grad()
        loss = torch.tensor(0.0, requires_grad=True).cuda()
        clf_output, reg_output, per_loss = self.model(inputs)
        clf_loss = self.criterion_clf(clf_output, labels)
        con_reg_group_loss = self.get_con_reg_group_loss.apply(reg_output, demors, self.frequency_dict, labels)
        loss = loss + clf_loss + reg_loss + torch.mean(con_reg_group_loss) + torch.mean(per_loss)
        loss.backward()
        self.optimizer.step()

In [7]:
optimizer = optim.Adam(model.parameters(), lr= 0.0001, betas=(0.5, 0.999))

criterion_clf = nn.CrossEntropyLoss(weight=torch.Tensor([1, 3])).to('mps')
criterion_reg = nn.SmoothL1Loss(reduction='mean').to('mps')

optimal_valid_matrix = [[0, 0], [0, 0]]
optimal_valid_metric = 0
optimal_epoch = 0

epoch = 0
train_epoch = 300

while epoch < train_epoch:
    self.train_model_epoch()

    print('{}th epoch validation confusion matrix:'.format(self.epoch), valid_matrix)
    print('eval_metric:', "%.4f" % self.eval_metric(valid_matrix), 'and mean squared error ', valid_mse)
    self.save_checkpoint(valid_matrix, valid_mse)
    self.epoch += 1


In [4]:
model

_CNN(
  (block_c_1): ConvLayer(
    (conv): Conv3d(1, 10, kernel_size=(7, 7, 7), stride=(2, 2, 2), bias=False)
    (pooling): MaxPool3d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (BN): BatchNorm3d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): LeakyReLU(negative_slope=0.01)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (block_c_2): ConvLayer(
    (conv): Conv3d(10, 20, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2), bias=False)
    (pooling): MaxPool3d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (BN): BatchNorm3d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): LeakyReLU(negative_slope=0.01)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (block_c_3): ConvLayer(
    (conv): Conv3d(40, 40, kernel_size=(5, 5, 5), stride=(1, 1, 1), bias=False)
    (pooling): MaxPool3d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (BN): BatchNorm3d(40